In [2]:
from datetime import date, timedelta
import os
import pickle
import sys
sys.path.append("..")

In [3]:
from abautomator import experiment

In [4]:
import pandas as pd


In [5]:
# %pip install -U pandas

In [42]:
name = "rel_diff_ci"
df = pickle.load(
    open(os.path.join("..", "tests", f"{name}.p"), "rb" )
)

In [43]:
df.head(2)

,exp_cond,metric,tx_mean,tx_std,tx_count,factor_label,ctrl_mean,ctrl_std,ctrl_count,abs_mean,abs_std,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci,mean,std
0,Video01,n_user_sessions,2.250142,2.785179,3514.0,"(n_user_sessions, Video01)",2.142777,2.302348,3572.0,0.107365,0.060758,7.846029,2.175091,10.681498,-0.660378,5.010560,2.835469
1,Video01,pct_user_sessions,0.866249,0.340433,3514.0,"(pct_user_sessions, Video01)",0.864222,0.342601,3572.0,0.002028,0.008114,1.173517,-0.704294,2.112422,-1.643199,0.234612,0.938905


In [44]:
import math

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Colorblind7
from bokeh.transform import factor_cmap

from math import pi

output_notebook()

Loading BokehJS ...

In [45]:
source = ColumnDataSource(df)

p = figure(
    y_range=FactorRange(*list(source.data["factor_label"])),
    height=450,
    title=f"exp name",
    toolbar_location=None,
    tools=""
)

p.segment(
    source.data["upper_95_ci"],
    source.data["factor_label"],
    source.data["upper_68_ci"],
    source.data["factor_label"],
    color="black",
)

p.segment(
    source.data["lower_95_ci"],
    source.data["factor_label"],
    source.data["lower_68_ci"],
    source.data["factor_label"],
    color="black",
)

p.hbar(
    y='factor_label',
    right="upper_68_ci",
    left="lower_68_ci",
    legend_field='exp_cond',
    fill_color=factor_cmap(
        'exp_cond', palette=Colorblind7, factors=list(set(source.data["exp_cond"])), start=1, end=2
    ),
    line_color=None,
    height=0.6,
    fill_alpha=0.8,
    source=source,
)

p.legend.location = "top_right"

p.ygrid.grid_line_color = None
p.xaxis.axis_label = 'Absolute difference from control'

p.yaxis.major_label_text_alpha = 0.0
p.yaxis.major_tick_in = 0
p.yaxis.major_tick_out = 0
p.yaxis.major_label_text_font_size = '1px'

p.yaxis.group_label_orientation = "horizontal"
p.yaxis.separator_line_alpha = 0
p.yaxis.minor_tick_line_color = "yellow"

In [46]:
show(p)